## Imports

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing

## Functions

In [ ]:
def fit_predict(univariate_train, max_horizon):
    fit0 = SimpleExpSmoothing(univariate_train).fit()
    fit1 = ExponentialSmoothing(univariate_train, seasonal_periods=24, trend='add', seasonal='add', use_boxcox=True, initialization_method="estimated").fit()
    fit2 = ExponentialSmoothing(univariate_train, seasonal_periods=24, trend='add', seasonal='mul', use_boxcox=True, initialization_method="estimated").fit()
    fit3 = ExponentialSmoothing(univariate_train, seasonal_periods=24, trend='add', seasonal='add', damped_trend=True, use_boxcox=True, initialization_method="estimated").fit()
    fit4 = ExponentialSmoothing(univariate_train, seasonal_periods=24, trend='add', seasonal='mul', damped_trend=True, use_boxcox=True, initialization_method="estimated").fit()
    
    fcast0 = fit0.forecast(max_horizon).rename("SimpleExpSmoothing")
    fcast1 = fit1.forecast(max_horizon).rename("Holt-Winters (add-add-seasonal)")
    fcast2 = fit2.forecast(max_horizon).rename("Holt-Winters (add-mul-seasonal)")
    fcast3 = fit3.forecast(max_horizon).rename("Holt-Winters (add-add-seasonal) - damp")
    fcast4 = fit4.forecast(max_horizon).rename("Holt-Winters (add-mul-seasonal) - damp")
   
    return pd.concat([fcast0,fcast1,fcast2,fcast3,fcast4],axis=1)

In [ ]:
def execute(series_name, time_series, test_parts, num_test_days, max_horizon):
    window_size = len(time_series) // test_parts
    all_points_h1 = pd.DataFrame(columns=np.arange(5))
    all_points_h2 = pd.DataFrame(columns=np.arange(5))
    for i in range(test_parts): 
        considered_slice = time_series[: window_size * (i+1)]
        for j in range(num_test_days+1): #0.......17 incluso
            train = considered_slice[: - 1 - num_test_days + j] #-18.....-1 (escluso)
           
            ret = fit_predict(train, max_horizon)
            h1 = ret.iloc[0,:]
            h2 = ret.iloc[1,:]
            if j > 0:
                all_points_h1.loc[len(all_points_h1)] = h1.values #-17.....-1 (incluso)
            if j < num_test_days:
                all_points_h2.loc[len(all_points_h2)] = h2.values #-18.....-2 (incluso)
    build_test = []
    for i in range(test_parts): 
        considered_slice = time_series[: window_size * (i+1)]
        test = considered_slice[- num_test_days:].values
        
        build_test.append(test)
    truth =  np.append(np.array([]),build_test)
    all_points_h1['Truth'] = truth
    all_points_h2['Truth'] = truth
    
    return all_points_h1, all_points_h2

## Load unpreprocessed data

In [ ]:
exchange = pd.read_csv("./../multivariate-time-series-data/exchange_rate/exchange_rate.txt",header=None)
electricity = pd.read_csv("./../multivariate-time-series-data/electricity/electricity.txt",header=None)

## Execution

In [ ]:
max_horizon = 2
test_parts = 3
test_size = 17 #total of test_parts * test_size data points in test_parts different moments of time 

datasets_names = ["Exchange"]
datasets = [exchange[0].clip(lower=0.000001)]
for index, dataset in enumerate(datasets):
    dataset_name = datasets_names[index]
    c3, c4 = execute(dataset_name, dataset, test_parts, test_size, max_horizon)

datasets_names = ["Electricity"]
datasets = [electricity[0].clip(lower=0.000001)]
for index, dataset in enumerate(datasets):
    dataset_name = datasets_names[index]
    c1, c2 = execute(dataset_name, dataset, test_parts, test_size, max_horizon)

## Saving results

In [ ]:
names = ["SimpleExpSmoothing","Holt-Winters (add-add-seasonal)","Holt-Winters (add-mul-seasonal)","Holt-Winters (add-add-seasonal) - damp","Holt-Winters (add-mul-seasonal) - damp","Truth"]
c1.columns = names
c2.columns = names
c3.columns = names
c4.columns = names

c1.to_csv("Electricity_h1_exponentialsmoothing.csv", index=False)
c2.to_csv("Electricity_h2_exponentialsmoothing.csv", index=False)
c3.to_csv("Exchange_h1_exponentialsmoothing.csv", index=False)
c4.to_csv("Exchange_h2_exponentialsmoothing.csv", index=False)